In [1]:
import torch
import torch.utils.benchmark as benchmark

from alibi_generator import ALiBiAttentionMaskGenerator
from metaseq import buffered_future_mask

In [2]:
seq_len = 512
batch_size = 1

t0 = benchmark.Timer(
    stmt = "buffered_future_mask(x)",
    globals = {
        "buffered_future_mask": buffered_future_mask,
        "x": torch.empty((seq_len, batch_size))
        },
    label = "Metaseq")

model_size = 1

t1 = benchmark.Timer(
    stmt = """
        alibi_gen = ALiBiAttentionMaskGenerator(num_heads=8)
        alibi_gen(x)
        """,
    globals = {
        "ALiBiAttentionMaskGenerator": ALiBiAttentionMaskGenerator,
        "x": torch.empty((seq_len, model_size))
        },
    label = "Fairseq2")

In [4]:
res0 = t0.timeit(10000)
res1 = t1.timeit(10000)
print(res0)
print(res1)

Metaseq
  861.32 us
  1 measurement, 10000 runs , 1 thread
Fairseq2
  708.30 us
  1 measurement, 10000 runs , 1 thread
